# 4. Regresión logística para predecir supervivientes en accidentes 

### CONTEXTO

Para la realización de este ejercicio simularemos que trabajas en el INE y que a partir de los datos de un dataset de accidentes vas a predecir la cantidad de supervivientes de accidentes en función del año, género, la velocidad del impacto y el uso de casco.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss

In [11]:
# Cargamos el CSV
df = pd.read_csv("accident.csv")

# Analizamos el contenido del mismo
print(df.info())
print(df.describe())

# Eliminamos los registros con valores nulos
df = df.dropna()

# Imprimimos los valores únicos de cada variable para determinar su tipo (categórica o numérica)
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              200 non-null    int64  
 1   Gender           199 non-null    object 
 2   Speed_of_Impact  197 non-null    float64
 3   Helmet_Used      200 non-null    object 
 4   Seatbelt_Used    200 non-null    object 
 5   Survived         200 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 9.5+ KB
None
             Age  Speed_of_Impact   Survived
count  200.00000       197.000000  200.00000
mean    43.42500        70.441624    0.50500
std     14.94191        30.125298    0.50123
min     18.00000        20.000000    0.00000
25%     31.00000        43.000000    0.00000
50%     43.50000        71.000000    1.00000
75%     56.00000        95.000000    1.00000
max     69.00000       119.000000    1.00000
Age: [56 69 46 32 60 25 38 36 40 28 41 53 57 20 39 19 61

Determinamos que las variables numéricas son ``Age`` y ``Speed_of_Impact`` y las categóricas son ``Gender``, ``Helmet_Used`` y ``Seatbelt_Used``. <br>
Nuestra variable a predecir es ``Survived``

In [15]:
# Encodeamos las variables categóricas mediante OneHot Encoder. Usamos el parámetro 'drop_first' para eliminar la primera columna generada y evitar redundancias
categoricas = ["Gender", "Helmet_Used", "Seatbelt_Used"]
for cat in categoricas:
    df[cat] = pd.get_dummies(df[cat], drop_first=True)

# Elegimos la variables predictoras junto con la variable a predecir
X = df.drop(columns=["Survived"])
y = df["Survived"]

# Preprocesamos las variables numéricas mediante el StandarScaler para que esten en la misma escala
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --------------------
# MÉTODOS DE BALANCEO DE CLASES
# --------------------
# 1. Sobremuestreo aleatorio
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 3. Submuestreo aleatorio
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# 4. NearMiss (submuestreo basado en distancia)
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

# --------------------
# EVALUACIÓN DE MÉTODOS DE BALANCEO
# --------------------
resultados_balanceo = {}

def evaluar_balanceo(X_train_resampled, y_train_resampled, metodo):
    # Creación del modelo de RL
    model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)
    # Entreno el modelo con los datos de cada modelo
    model.fit(X_train_resampled, y_train_resampled)
    # Realizo predicciones con el modelo
    y_pred = model.predict(X_test)
    # Calculo la precisión del modelo sacando las métricas
    f1 = f1_score(y_test, y_pred) * 100
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) * 100
    resultados_balanceo[metodo] = (f1, auc)
    # Imprimimos los resultados
    print(f"\nResultados con {metodo}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred) * 100:.2f}%")
    print(f"F1-Score: {f1:.2f}%")
    print(f"AUC-ROC: {auc:.2f}%")

evaluar_balanceo(X_train_ros, y_train_ros, "Sobremuestreo Aleatorio")
evaluar_balanceo(X_train_smote, y_train_smote, "SMOTE")
evaluar_balanceo(X_train_rus, y_train_rus, "Submuestreo Aleatorio")
evaluar_balanceo(X_train_nm, y_train_nm, "NearMiss")

# Seleccionar el mejor método de balanceo
mejor_metodo = max(resultados_balanceo, key=lambda k: resultados_balanceo[k])
print(f"\nEl mejor método de balanceo es: {mejor_metodo} con F1-Score: {resultados_balanceo[mejor_metodo][0]:.2f}% y AUC-ROC: {resultados_balanceo[mejor_metodo][1]:.2f}%")

# Obtener los datos balanceados del mejor método encontrado
if mejor_metodo == "Sobremuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_ros, y_train_ros
elif mejor_metodo == "SMOTE":
    X_train_resampled, y_train_resampled = X_train_smote, y_train_smote
elif mejor_metodo == "Submuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_rus, y_train_rus
elif mejor_metodo == "NearMiss":
    X_train_resampled, y_train_resampled = X_train_nm, y_train_nm

# --------------------
# MÉTODO 1: Ajuste manual de hiperparámetros
# --------------------
manual_model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)  # Ejemplo de hiperparámetros ajustados manualmente
manual_model.fit(X_train_resampled, y_train_resampled)  # Entrenar modelo

# Predicciones
y_pred_manual = manual_model.predict(X_test)

# Evaluación del modelo ajustado manualmente
print("\nResultados con ajuste manual de hiperparámetros:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, manual_model.predict_proba(X_test)[:,1]) * 100:.2f}%")

# --------------------
# MÉTODO 2: Ajuste automático con GridSearchCV
# --------------------
parametros = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],  # Diferentes valores de regularización
    "solver": ["liblinear", "lbfgs"]  # Diferentes algoritmos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=500), parametros, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)  # Entrenar búsqueda de hiperparámetros

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Evaluación del mejor modelo
print("\nResultados con ajuste automático de hiperparámetros:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor Accuracy en validación cruzada: {grid_search.best_score_ * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_best) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_best) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) * 100:.2f}%")


# --------------------
# ENTRENAMIENTO FINAL Y EVALUACIÓN
# --------------------

# Entrenar el mejor modelo con los datos balanceados
best_model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones finales con el mejor modelo y el mejor balanceo
y_pred_final = best_model.predict(X_test)

# Crear DataFrame de resultados
resultados = pd.DataFrame({
    "Real": y_test.values,
    "Predicción": y_pred_final
})

# Mapear valores 0 y 1 a etiquetas comprensibles
resultados["Real"] = resultados["Real"].map({0: "Fallecido", 1: "Vivo"})
resultados["Predicción"] = resultados["Predicción"].map({0: "Fallecido", 1: "Vivo"})

# Mostrar las primeras filas de la tabla
print(resultados.head(10))



Resultados con Sobremuestreo Aleatorio:
Accuracy: 50.00%
Precision: 40.00%
Recall: 66.67%
F1-Score: 50.00%
AUC-ROC: 53.33%

Resultados con SMOTE:
Accuracy: 55.00%
Precision: 43.48%
Recall: 66.67%
F1-Score: 52.63%
AUC-ROC: 52.80%

Resultados con Submuestreo Aleatorio:
Accuracy: 60.00%
Precision: 47.83%
Recall: 73.33%
F1-Score: 57.89%
AUC-ROC: 56.53%

Resultados con NearMiss:
Accuracy: 55.00%
Precision: 43.48%
Recall: 66.67%
F1-Score: 52.63%
AUC-ROC: 60.53%

El mejor método de balanceo es: Submuestreo Aleatorio con F1-Score: 57.89% y AUC-ROC: 56.53%

Resultados con ajuste manual de hiperparámetros:
Accuracy: 60.00%
Precision: 47.83%
Recall: 73.33%
F1-Score: 57.89%
AUC-ROC: 56.53%

Resultados con ajuste automático de hiperparámetros:
Mejores parámetros: {'C': 0.01, 'solver': 'liblinear'}
Mejor Accuracy en validación cruzada: 54.93%
Accuracy: 57.50%
Precision: 45.45%
Recall: 66.67%
F1-Score: 54.05%
AUC-ROC: 54.93%
        Real Predicción
0  Fallecido  Fallecido
1  Fallecido  Fallecido
2  